##### Copyright 2025 Google LLC.

安装 Agent Development Kit (ADK)<br>
配置你的API密钥以使用国内大语言模型<br>
构建你的第一个简单智能体<br>
运行你的智能体，观察它使用工具（如Google搜索）来回答问题<br>

## 安装
安装依赖
```
pip install google-adk
```

配置模型API<br>
复制.env.example文件，重命名为.env<br>

In [1]:

import os
import dotenv

dotenv.load_dotenv()
print("✅ 模型API配置成功")

✅ 模型API配置成功


导入 ADK 组件

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini  # 使用Gemini模型
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search  # 使用谷歌搜索
from google.genai import types

print("✅ ADK组件导入成功")

✅ ADK组件导入成功


构建 AI 智能体

LLM语言模型：<br>
`提示词 -> LLM -> 文本`<br>
AI 智能体：<br>
`提示词 -> 智能体 -> 思考 -> 行动 -> 观察 -> 最终答案`<br>

定义智能体

name和description：一个简单的名称和描述，用于标识我们的智能体。<br>
model：将为智能体的推理提供动力的特定 LLM。我们将使用 "gemini-2.5-flash-lite"。<br>
instruction：智能体的指导性提示词。这告诉智能体它的目标是什么以及应该如何表现。<br>
tools：智能体可以使用的[工具](https://google.github.io/adk-docs/tools/)列表。首先，我们将给它 `google_search` 工具，这让它能够在线查找最新信息。

In [3]:
# 创建搜索工具

from duckduckgo_search import DDGS
import warnings
import datetime
warnings.filterwarnings("ignore")


def search_tools(query: str, top_k: int = 3) -> dict:
    results = [
        {
            "title": "2025热门电影",    
            "snippet": "2025年12月30日，2025热门电影上映，包括《流浪地球2》、《满江红》、《熊出没·伴我“熊芯”》等。",
            "metadata": "duckduckgo"
        },
        {
            "title": "2025热门电视剧",
            "snippet": "2025年12月30日，2025热门电视剧上映，包括《狂飙》、《三体》、《去有风的地方》等。",
            "metadata": "duckduckgo"
        }, 
    ]
    
    return results


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    current_time = datetime.datetime.now().strftime("%Y年%m月%d日 %H:%M:%S")
    return {"current_time": current_time}


In [4]:
# root_agent = Agent(
#     name="helpful_assistant",
#     model=Gemini(
#         model="gemini-2.5-flash-lite",
#         retry_options=retry_config
#     ),
#     description="A simple agent that can answer general questions.",
#     instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
#     tools=[google_search],
# )

from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
# 使用LiteLlm接口适配其他厂商模型

root_agent = Agent(
    name="helpful_assistant",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use web_search_free for current info or if unsure.",
    tools=[search_tools, get_current_time],
)
print("✅ 智能体已创建")

✅ 智能体已创建


运行智能体

Runner：管理对话，将我们的消息发送给智能体，并处理其响应。<br>
创建一个 `InMemoryRunner` 并配置 `root_agent`：<br>

In [5]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner 创建成功")

✅ Runner 创建成功


调用 `.run_debug()` 方法来发送提示词并获取答案

In [6]:
response = await runner.run_debug(
    "现在电影院正在上映哪些新电影？"
)


 ### Created new session: debug_session_id

User > 现在电影院正在上映哪些新电影？
helpful_assistant > 我来帮您查询当前正在上映的新电影信息。
helpful_assistant > 让我重新搜索更具体的信息：
helpful_assistant > 让我尝试搜索更具体的电影信息：
helpful_assistant > 根据搜索结果，我看到了一些信息，但搜索结果似乎主要显示的是2025年的热门电影信息。让我获取当前时间，以便为您提供更准确的信息：
helpful_assistant > 根据当前时间（2025年12月30日）和搜索结果，我为您整理了当前可能正在上映的电影信息：

## 近期可能正在上映的电影：

### 热门电影（基于搜索结果）：
1. **《流浪地球2》** - 科幻灾难片，中国科幻电影的代表作
2. **《满江红》** - 历史悬疑片，张艺谋导演作品
3. **《熊出没·伴我"熊芯"》** - 动画电影，适合家庭观看

### 其他可能正在热映的电影：
由于现在是2025年12月底，正值年末贺岁档，可能还有以下类型的电影：
- 贺岁喜剧片
- 动作大片
- 爱情片
- 动画电影

### 建议：
1. **查看本地影院APP**：建议您打开猫眼、淘票票或本地影院APP查看实时排片
2. **关注新片上映**：12月底通常会有多部新片上映迎接元旦假期
3. **查看影评网站**：豆瓣电影、时光网等网站会有最新的上映信息和评分

### 热门观影选择：
- **科幻爱好者**：《流浪地球2》
- **历史悬疑爱好者**：《满江红》
- **家庭观众**：《熊出没》系列
- **贺岁喜剧**：可能会有新的贺岁喜剧片上映

由于电影排片会因地区和影院而异，建议您直接查看当地影院的实时排片信息，以获得最准确的上映电影列表。


智能体如何工作的？

智能体执行了搜索以获取关于ADK的最新信息，它知道使用这个工具是因为：

1. 智能体会检查并了解它可以使用哪些工具。
2. 智能体的指令指定了使用搜索工具来获取最新信息，或者在不确定答案时使用。

查看智能体思考和行动的完整、详细追踪的最佳方式是在**ADK web UI**中，我们将在本notebook的后面部分设置它。

我们将在课程后面介绍更详细的日志记录和可观测性工作流程。

2.5 轮到你了！

这是你观察智能体实际运行的机会。向它提出一个需要最新信息的问题。

试试下面这些问题，或者自己想一个：

- 伦敦的天气怎么样？
- 谁赢得了上一届足球世界杯？
- 现在电影院正在上映哪些新电影？

In [7]:
response = await runner.run_debug("现在热度较高的电视剧有哪些？")


 ### Continue session: debug_session_id

User > 现在热度较高的电视剧有哪些？
helpful_assistant > 我来帮您查询当前热度较高的电视剧信息。
helpful_assistant > 让我搜索更具体的电视剧信息：
helpful_assistant > 让我搜索更多关于当前热播电视剧的信息：
helpful_assistant > 根据搜索结果，我为您整理了当前（2025年12月）热度较高的电视剧信息：

## 当前热度较高的电视剧：

### 热门剧集（基于搜索结果）：
1. **《狂飙》** - 刑侦题材，讲述扫黑除恶的故事，口碑和热度都很高
2. **《三体》** - 科幻巨制，改编自刘慈欣同名小说，制作精良
3. **《去有风的地方》** - 治愈系田园剧，讲述都市人回归自然的故事

### 其他可能正在热播的电视剧类型：
考虑到现在是2025年底，可能还有以下类型的剧集：

**可能的热门类型：**
- **古装剧**：年末通常有大型古装剧播出
- **都市情感剧**：适合年末观看的轻松剧集
- **悬疑推理剧**：一直很受欢迎的类型
- **职场剧**：反映当代职场生活的剧集

### 推荐观看平台：
1. **爱奇艺、腾讯视频、优酷**：三大主流视频平台
2. **芒果TV**：适合观看青春、都市类剧集
3. **B站**：有一些独播剧集和优质内容

### 热门剧集特点：
- **《狂飙》**：剧情紧凑，演员演技出色，社会话题性强
- **《三体》**：科幻迷必看，特效制作精良，忠实原著
- **《去有风的地方》**：节奏舒缓，画面优美，适合放松心情

### 建议：
1. **查看实时榜单**：建议查看各大视频平台的"热播榜"或"飙升榜"
2. **关注豆瓣评分**：豆瓣评分通常能反映剧集质量
3. **查看社交媒体讨论**：微博、小红书等平台的热门话题

### 不同观众推荐：
- **喜欢紧张刺激**：《狂飙》
- **科幻爱好者**：《三体》
- **想要放松治愈**：《去有风的地方》
- **喜欢古装**：可以关注最新的古装剧集

由于电视剧的播出和热度会随时间变化，建议您直接查看视频平台的最新榜单，以获得最准确的热度信息。



第3节：尝试 ADK Web 界面

ADK 包含一个内置的 Web 界面，用于与你的智能体进行交互式聊天、测试和调试。

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

要使用 ADK web UI，你需要使用 `adk create` 命令创建一个包含 Python 文件的智能体。

运行下面的命令来生成一个 `sample-agent` 文件夹，其中包含所有必要的文件，包括用于你的代码的 `agent.py`、预配置了 API 密钥的 `.env` 文件，以及一个 `__init__.py` 文件：

In [ ]:
!adk create sample_agent --model LiteLlm(model="deepseek/deepseek-chat") --api_key $DEEPSEEK_API_KEY


Agent created in d:\code\agent_learn\agent-course-google2025\Day1\sample-agent:
- .env
- __init__.py
- agent.py



Now we can run ADK web:

In [10]:
!adk web

^C


现在你可以使用上面的链接访问 ADK 开发 UI。

In [ ]:
# +-----------------------------------------------------------------------------+
# | ADK Web Server started                                                      |
# |                                                                             |
# | For local testing, access at http://127.0.0.1:8000.                         |
# +-----------------------------------------------------------------------------+